In [45]:
!pip3 install scikit-learn tqdm catboost optuna xgboost ipywidgets imblearn

In [46]:
import pandas as pd
import sklearn
import numpy as np
import random
import os
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner
from catboost import CatBoostClassifier, Pool

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold


In [47]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed=42
seed_everything(seed) # Seed 고정

In [48]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [49]:
def category_all(df_train, df_test):
    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    'customer_idx',
    'lead_owner',
        'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',

    ]
    for i in label_columns:
        df_train[i] = df_train[i].astype('category')
        df_test[i] = df_test[i].astype('category')
    return df_train, df_test    


def preprocessing(df_train, df_test):
    df_train.drop_duplicates(keep='first', inplace=True)
    df_train.reset_index(inplace=True)
    df_train.drop('index',inplace=True, axis=1)
    df_test_id_temp = df_test['id'].copy()
    df_all = pd.concat([df_train,df_test.drop('id',axis=1)])
    df_all.drop('customer_country.1',axis=1,inplace=True)
    df_all.id_strategic_ver.fillna(0., inplace=True)
    df_all.it_strategic_ver.fillna(0., inplace=True)
    df_all.idit_strategic_ver.fillna(0., inplace=True)
    df_all.business_subarea.fillna('NAN', inplace=True)
    df_all.com_reg_ver_win_rate.fillna(0,inplace=True)
    df_all['customer_type'] = df_all['customer_type'].replace({
        'End-Customer':'End Customer'
    })
    df_all.customer_type.fillna('NAN',inplace=True)
    df_all.historical_existing_cnt.fillna(0, inplace=True)
    
    df_all.product_subcategory.fillna('NAN', inplace=True)
    
    df_all.product_modelname.fillna('NAN', inplace=True)

    df_all['expected_timeline'] = df_all['expected_timeline'].replace({
        'less_than_3_months': 'less than 3 months',
        'being followed up': 'being followed up.',
        'less than 6 months': '3 months ~ 6 months',
        '3_months_~_6_months' : '3 months ~ 6 months'
    })

    df_all.expected_timeline.fillna('NAN',inplace=True)
    df_all.ver_win_rate_x.fillna(0,inplace=True)
    df_all.ver_win_ratio_per_bu.fillna(0,inplace=True)
    df_all.business_area.fillna('NAN',inplace=True)
    df_all.customer_job.fillna('NAN', inplace=True)
    df_all.product_category.fillna('NAN', inplace=True)
    
    #처리못한부분
    #customer_country
    df_all.customer_country.fillna('NAN', inplace=True)
    #inquiry_type
    df_all.inquiry_type.fillna('NAN', inplace=True)

    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    ]
    
    ###############################
    #detect special char and to lower
    for i in label_columns:
        df_all[i] = df_all[i].str.lower()
        df_all[i] = df_all[i].str.replace(pat=r'[^\w]', repl=r'', regex=True)
        
    #to int
    df_all['id_strategic_ver'] = df_all['id_strategic_ver'].astype('int64')
    df_all['it_strategic_ver'] = df_all['it_strategic_ver'].astype('int64')
    df_all['idit_strategic_ver'] = df_all['idit_strategic_ver'].astype('int64')
    df_all['lead_desc_length'] = df_all['lead_desc_length'].astype('int64')
    df_all['historical_existing_cnt'] = df_all['historical_existing_cnt'].astype('int64')
    
    #seperate
    tmp_len = len(df_train)
    df_train = df_all.iloc[: tmp_len ]
    df_test = df_all.iloc[tmp_len :]
    df_test = pd.concat([df_test,df_test_id_temp],axis=1)
    
    #RobustScaler
    robustScaler = RobustScaler()
    df_train['ver_win_rate_x'] = robustScaler.fit_transform(df_train[['ver_win_rate_x']])
    df_test['ver_win_rate_x'] = robustScaler.transform(df_test[['ver_win_rate_x']])
    robustScaler = RobustScaler()
    df_train['ver_win_ratio_per_bu'] = robustScaler.fit_transform(df_train[['ver_win_ratio_per_bu']])
    df_test['ver_win_ratio_per_bu'] = robustScaler.transform(df_test[['ver_win_ratio_per_bu']])
    ##############################
    
    return df_train, df_test


df_train, df_test = preprocessing(df_train, df_test)
df_train, df_test = category_all(df_train, df_test)
rate = ((len(df_train)-sum(df_train['is_converted']))/sum(df_train['is_converted']))
x_train, y_train = df_train.drop("is_converted",axis=1), df_train["is_converted"]

In [52]:
categorical_features = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
   'customer_idx',
   'lead_owner',
    'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',
    ]

S_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

all_train_pool = []
all_val_pool = []
all_y_val = []
for train_idx, val_idx in S_kfold.split(x_train, list(y_train)):
    tmp_x_train, tmp_x_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    tmp_y_train, tmp_y_val = y_train[train_idx], y_train[val_idx]
    train_pool = Pool(data=tmp_x_train, label=tmp_y_train, cat_features=categorical_features)
    val_pool = Pool(data=tmp_x_val, label=tmp_y_val, cat_features=categorical_features)
    all_train_pool.append(train_pool)
    all_val_pool.append(val_pool)
    all_y_val.append(list(tmp_y_val))

In [54]:
def objective(trial):
    params = {'iterations':trial.suggest_int("iterations", 1000, 50000),
        'learning_rate' : trial.suggest_float('learning_rate',0.001, 1),
        'reg_lambda': trial.suggest_float('reg_lambda',10,100),
        'random_strength': trial.suggest_float('random_strength',40,100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_float('bagging_temperature', 0, 1),
             }
    
    #score = []
    tmp_score = []
    for train_pool, val_pool, y_true in zip(all_train_pool, all_val_pool, all_y_val):
        clf = CatBoostClassifier(**params,
                                 scale_pos_weight=rate, 
                                 loss_function='Logloss',
                                 eval_metric='F1',
                                 early_stopping_rounds=100,
                                 verbose=False,
                                random_state=seed,
                                 task_type="GPU",
                                 devices='0',
                                 max_ctr_complexity=15
                                 
                                )
        clf.fit(train_pool, eval_set=val_pool)
        
        y_pred = clf.predict(val_pool)
        y_pred = [False if i=='False' else True for i in y_pred]
        f1 = f1_score(y_true,y_pred,labels=[True, False])
        tmp_score.append(f1 if f1>0.77 else 0)
    tmp_score = np.mean(tmp_score)
    return tmp_score

# Hyperparameter Tuning
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=seed), pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=None)

[I 2024-02-17 11:28:16,085] A new study created in memory with name: no-name-01e06183-84c9-43b7-a088-afc4b682af14
[I 2024-02-17 11:28:45,970] Trial 0 finished with value: 0.15539118065433855 and parameters: {'iterations': 19352, 'learning_rate': 0.9507635921035062, 'reg_lambda': 75.87945476302646, 'random_strength': 75.91950905182219, 'min_data_in_leaf': 8, 'leaf_estimation_iterations': 3, 'bagging_temperature': 0.05808361216819946}. Best is trial 0 with value: 0.15539118065433855.
[I 2024-02-17 11:29:15,683] Trial 1 finished with value: 0.3119637454882068 and parameters: {'iterations': 43443, 'learning_rate': 0.6015138967314656, 'reg_lambda': 73.7265320016441, 'random_strength': 41.23506965774815, 'min_data_in_leaf': 49, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.21233911067827616}. Best is trial 1 with value: 0.3119637454882068.
[I 2024-02-17 11:30:01,236] Trial 2 finished with value: 0.31152417237215707 and parameters: {'iterations': 9909, 'learning_rate': 0.18422110

[I 2024-02-17 11:43:31,081] Trial 22 finished with value: 0.3131974214421451 and parameters: {'iterations': 30723, 'learning_rate': 0.5327320311181336, 'reg_lambda': 10.447957221338543, 'random_strength': 65.65688497906464, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 6, 'bagging_temperature': 0.6504877043425837}. Best is trial 13 with value: 0.547131881830636.
[I 2024-02-17 11:44:08,581] Trial 23 finished with value: 0.3900997318488689 and parameters: {'iterations': 23652, 'learning_rate': 0.295370251254916, 'reg_lambda': 28.99571967330085, 'random_strength': 59.706990237424804, 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.49923001001907086}. Best is trial 13 with value: 0.547131881830636.
[I 2024-02-17 11:44:41,653] Trial 24 finished with value: 0.5458792866471858 and parameters: {'iterations': 37394, 'learning_rate': 0.47009404650736336, 'reg_lambda': 19.353247337314393, 'random_strength': 54.9152064029765, 'min_data_in_leaf': 19, 'leaf

[I 2024-02-17 11:58:42,502] Trial 44 finished with value: 0.5472734311070131 and parameters: {'iterations': 37204, 'learning_rate': 0.4942464465812314, 'reg_lambda': 74.51416141225258, 'random_strength': 56.408950652993326, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.3980767409179816}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 11:59:11,788] Trial 45 finished with value: 0.1566154200974747 and parameters: {'iterations': 33762, 'learning_rate': 0.919796778341636, 'reg_lambda': 73.83997105551762, 'random_strength': 56.78898196913975, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.4078181315296423}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 12:00:04,421] Trial 46 finished with value: 0.46939425964812315 and parameters: {'iterations': 29373, 'learning_rate': 0.1807892391841041, 'reg_lambda': 82.13464784471118, 'random_strength': 61.78905575255391, 'min_data_in_leaf': 50, 'lea

[I 2024-02-17 12:12:35,361] Trial 66 finished with value: 0.31444021634017794 and parameters: {'iterations': 31635, 'learning_rate': 0.5120468096648393, 'reg_lambda': 19.24481606006373, 'random_strength': 70.25372708968378, 'min_data_in_leaf': 25, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.7333563685849502}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 12:13:00,366] Trial 67 finished with value: 0.0 and parameters: {'iterations': 39796, 'learning_rate': 0.9957310404548301, 'reg_lambda': 24.578368237786325, 'random_strength': 55.18382010576914, 'min_data_in_leaf': 45, 'leaf_estimation_iterations': 7, 'bagging_temperature': 0.6540678027553207}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 12:13:29,468] Trial 68 finished with value: 0.07775800711743772 and parameters: {'iterations': 34264, 'learning_rate': 0.6219225103572058, 'reg_lambda': 61.818185840162826, 'random_strength': 77.45994516724063, 'min_data_in_leaf': 20, 'leaf_estimation_i

[I 2024-02-17 12:26:19,434] Trial 88 finished with value: 0.23692218616197297 and parameters: {'iterations': 43429, 'learning_rate': 0.39079466859756407, 'reg_lambda': 18.667145878635914, 'random_strength': 76.8526581771613, 'min_data_in_leaf': 19, 'leaf_estimation_iterations': 6, 'bagging_temperature': 0.6720559846508205}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 12:26:54,123] Trial 89 finished with value: 0.39202411419913324 and parameters: {'iterations': 34499, 'learning_rate': 0.44432016840519806, 'reg_lambda': 35.26446589887841, 'random_strength': 72.62984528806706, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 4, 'bagging_temperature': 0.29303829652590235}. Best is trial 33 with value: 0.7042084772533082.
[I 2024-02-17 12:27:33,256] Trial 90 finished with value: 0.467928527698556 and parameters: {'iterations': 29704, 'learning_rate': 0.28374946491705333, 'reg_lambda': 30.10362606184577, 'random_strength': 68.51317937669404, 'min_data_in_leaf': 14, 'l

...

[I 2024-02-18 01:31:50,177] Trial 1298 finished with value: 0.39037791764966473 and parameters: {'iterations': 29618, 'learning_rate': 0.3389183286215228, 'reg_lambda': 32.75316092829584, 'random_strength': 88.34635740595738, 'min_data_in_leaf': 29, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.0013421062315363188}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 01:32:28,446] Trial 1299 finished with value: 0.3947894797830123 and parameters: {'iterations': 28168, 'learning_rate': 0.31683856724244297, 'reg_lambda': 43.1621467870946, 'random_strength': 86.2861186235366, 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.05623740372590072}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 01:33:06,231] Trial 1300 finished with value: 0.39727290548719213 and parameters: {'iterations': 25741, 'learning_rate': 0.3605366242460236, 'reg_lambda': 36.572201198967484, 'random_strength': 84.73451460616795, 'min_data_in_le

[I 2024-02-18 01:46:28,364] Trial 1320 finished with value: 0.23537203160822112 and parameters: {'iterations': 27628, 'learning_rate': 0.47024780148578654, 'reg_lambda': 63.908201636334574, 'random_strength': 77.61634938081417, 'min_data_in_leaf': 29, 'leaf_estimation_iterations': 9, 'bagging_temperature': 0.18416753149716472}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 01:47:05,700] Trial 1321 finished with value: 0.6236565770439466 and parameters: {'iterations': 26988, 'learning_rate': 0.4142910426052238, 'reg_lambda': 64.84112100803749, 'random_strength': 79.84551407653953, 'min_data_in_leaf': 29, 'leaf_estimation_iterations': 9, 'bagging_temperature': 0.18847702199952981}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 01:47:40,797] Trial 1322 finished with value: 0.4695699268920686 and parameters: {'iterations': 28105, 'learning_rate': 0.4077081084136076, 'reg_lambda': 61.27145272479144, 'random_strength': 78.63196041314193, 'min_data_in_leaf'

[I 2024-02-18 02:00:27,340] Trial 1342 finished with value: 0.3133601657812551 and parameters: {'iterations': 23900, 'learning_rate': 0.3508349738507048, 'reg_lambda': 28.44328720177039, 'random_strength': 74.70200722986075, 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.273279800153363}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 02:01:07,026] Trial 1343 finished with value: 0.6248753618832621 and parameters: {'iterations': 25088, 'learning_rate': 0.3686882548879155, 'reg_lambda': 30.862345958388353, 'random_strength': 76.69767706348223, 'min_data_in_leaf': 27, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.2268254611804071}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 02:01:47,297] Trial 1344 finished with value: 0.6299969650623852 and parameters: {'iterations': 24935, 'learning_rate': 0.3901062702581108, 'reg_lambda': 43.79510259790618, 'random_strength': 73.91874389455823, 'min_data_in_leaf': 3

[I 2024-02-18 02:14:29,569] Trial 1364 finished with value: 0.31550998943613856 and parameters: {'iterations': 39475, 'learning_rate': 0.3603943637143202, 'reg_lambda': 37.11999918584825, 'random_strength': 90.1807720243258, 'min_data_in_leaf': 33, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.2049986162427222}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 02:15:08,907] Trial 1365 finished with value: 0.4678012124136763 and parameters: {'iterations': 28001, 'learning_rate': 0.3509019644931494, 'reg_lambda': 41.35771734959898, 'random_strength': 89.82419746097091, 'min_data_in_leaf': 32, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.18386147771040276}. Best is trial 813 with value: 0.7834885296810342.
[I 2024-02-18 02:15:43,848] Trial 1366 finished with value: 0.39261129616416257 and parameters: {'iterations': 28291, 'learning_rate': 0.39859676151345413, 'reg_lambda': 49.4573488890575, 'random_strength': 89.32386046715946, 'min_data_in_leaf':

KeyboardInterrupt: 

In [65]:
best_param = study.best_params
best_param = {'iterations': 24282,
 'learning_rate': 0.110871449739074,
 'reg_lambda': 76.36512708631427,
 'random_strength': 63.92234212118737,
 'min_data_in_leaf': 5,
 'leaf_estimation_iterations': 14,
 'bagging_temperature': 0.638318444544851}

In [66]:
best_models = []
for train_pool, val_pool in zip(all_train_pool, all_val_pool):
    best_model = CatBoostClassifier(**best_param,
                                 random_state=seed,
                                 scale_pos_weight=rate, 
                                 loss_function='Logloss',
                                 eval_metric='F1',
                                 task_type="GPU",
                                 devices='0',
                                verbose=False,
                                   early_stopping_rounds=100,
                                   )
    
    best_model.fit(train_pool, eval_set=val_pool)
    best_models.append(best_model)

In [67]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [68]:
train_pred = [best_model.predict_proba(df_train.drop("is_converted",axis=1)) for best_model in best_models]
y = list(df_train['is_converted'])
train_pred = np.mean(train_pred[:],axis=0)
threshold = min([num[1] if y[i] else 1 for i, num in enumerate(train_pred)])
train_y_pred = [True if i[1]>=0.5 else False for i in train_pred]
get_clf_eval(y, train_y_pred)

오차행렬:
 [[ 4608    12]
 [ 1157 50003]]

정확도: 0.9790
정밀도: 0.7993
재현율: 0.9974
F1: 0.8874


In [74]:
threshold

0.09897288416696795

In [69]:
categorical_features =  [
                        "customer_country",
                        "business_subarea",
                        "business_area",
                        "business_unit",
                        "customer_type",
                        "enterprise",
                        "customer_job",
                        "inquiry_type",
                        "product_category",
                        "product_subcategory",
                        "product_modelname",
                        "customer_position",
                        "response_corporate",
                        "expected_timeline",
                        'customer_idx',
                        'lead_owner',
     'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',
                    ]
# 
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)
test_pool = Pool(data=x_test, cat_features=categorical_features)


In [70]:
# 예측 수행 (soft voting)
a_pred = [best_model.predict_proba(test_pool) for best_model in best_models]
pred = np.mean(a_pred[:], axis=0)
#pred = [True if i[1]>=threshold else False for i in pred]
pred = np.argmax(pred, axis=1)
sum(pred)

1822

In [71]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [73]:
real = pd.read_csv(r'submission(0.7707865168539325).csv')
get_clf_eval(real['is_converted'], pred)

오차행렬:
 [[1739   94]
 [  83 3355]]

정확도: 0.9664
정밀도: 0.9544
재현율: 0.9487
F1: 0.9516
